# Create the data based on the following description



In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances
from scipy.sparse import csr_matrix
import random


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from loss import nll

In [30]:

def build_item_attribute_matrix(item_attribute_dict, attribute_list=None):
    items = list(item_attribute_dict.keys())
    attributes = attribute_list if attribute_list else sorted({a for attrs in item_attribute_dict.values() for a in attrs})
    item_idx = {item: i for i, item in enumerate(items)}
    attr_idx = {attr: i for i, attr in enumerate(attributes)}
    matrix = np.zeros((len(items), len(attributes)))
    
    for item, attrs in item_attribute_dict.items():
        for attr in attrs:
            matrix[item_idx[item], attr_idx[attr]] = 1
    return matrix, items, attributes


def jaccard_similarity(matrix):
    A = matrix.astype(bool).astype(int)  # Ensure binary matrix
    n = A.shape[1]
    sim = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i == j:
                sim[i, j] = 1.0
            else:
                inter = np.logical_and(A[:, i], A[:, j]).sum()
                union = np.logical_or(A[:, i], A[:, j]).sum()
                sim[i, j] = inter / union if union != 0 else 0.0
    return sim


def generate_similarity_based_user_attributes(attr_sim, num_users, top_k=5, diversity_rounds=2):
    num_attributes = attr_sim.shape[0]
    user_attr_matrix = np.zeros((num_users, num_attributes))
    
    for u in range(num_users):
        selected = set()
        for _ in range(diversity_rounds):
            remaining_attrs = [a for a in range(num_attributes) if a not in selected]
            if not remaining_attrs:
                break
            seed = random.choice(remaining_attrs)
            sims = attr_sim[seed]
            probs = sims / sims.sum()
            non_zero_indices = np.where(probs > 0)[0]
            sample_size = min(top_k, len(non_zero_indices))
            chosen = np.random.choice(non_zero_indices, size=sample_size, replace=False)
            selected.update(chosen)
        user_attr_matrix[u, list(selected)] = 1
    return user_attr_matrix

def compute_attribute_overlap(matrix):
    I = matrix
    overlaps = {}
    for a in range(I.shape[1]):
        for b in range(a+1, I.shape[1]):
            Ia, Ib = I[:, a], I[:, b]
            inter = np.logical_and(Ia, Ib).sum()
            if Ia.sum() == 0 or Ib.sum() == 0: continue
            rmax = max(inter / Ia.sum(), inter / Ib.sum())
            rmin = min(inter / Ia.sum(), inter / Ib.sum())
            overlaps[(a, b)] = (rmax, rmin)
    return overlaps

def generate_set_theoretic_rules(overlaps, num_users, k=2, l=1, tau1=0.8, tau2=0.2, tau3=0.85):
    user_rules = []
    conjunctions = [pair for pair, (rmax, rmin) in overlaps.items() if rmax < tau1 and rmin > tau2]
    negations = [pair for pair, (rmax, rmin) in overlaps.items() if rmin > tau3]

    for _ in range(num_users):
        num_and_clauses = random.randint(2, 4)
        num_not_clauses = random.randint(0, 1)
        
        P = random.sample(conjunctions, min(num_and_clauses, len(conjunctions)))
        N = random.sample(negations, min(num_not_clauses, len(negations)))
        clause = {"and": P, "not": N}
        user_rules.append(clause)
    return user_rules, conjunctions, negations

def evaluate_dnf_rule(rule, item_attrs):
    # example rule: {'and': [(0, 1), (2, 3)], 'not': [(4, 5)]}
    # need to validate each and and each not rule
    # for this example, we check of 0 and 1, 2 and 3 are present, and 4 is present but 5 is not present.
    for and_clause in rule['and']:
        if all(item_attrs[a] for a in and_clause):
            return 1
    
    for present, not_present in rule['not']:
        if item_attrs[present] and not item_attrs[not_present]:
            return 1

    return 0


def generate_user_item_matrix_dot(Mua, Mia, tau_user=None):
    scores = Mua @ Mia.T
    Mui_dot = np.zeros_like(scores)
    for u in range(scores.shape[0]):
        threshold = tau_user[u] if tau_user is not None else np.percentile(scores[u], 75)
        Mui_dot[u] = (scores[u] > threshold).astype(int)
    return Mui_dot

def generate_user_item_matrix_set(user_rules, Mia):
    num_users = len(user_rules)
    num_items = Mia.shape[0]
    Mui_set = np.zeros((num_users, num_items))
    for u in range(num_users):
        for i in range(num_items):
            Mui_set[u, i] = evaluate_dnf_rule(user_rules[u], Mia[i])
    return Mui_set

def combine_preferences(Mui_dot, Mui_set, p=0.5):
    U = Mui_dot.shape[0]
    z = np.random.binomial(1, p, U)
    return np.array([Mui_set[u] if z[u] else Mui_dot[u] for u in range(U)])

def apply_noise(matrix, noise_fraction=0.1):
    noisy_matrix = matrix.copy()
    indices = np.argwhere(matrix == 1)
    num_flips = int(noise_fraction * len(indices))
    to_flip = indices[np.random.choice(len(indices), num_flips, replace=False)]
    for i, j in to_flip:
        noisy_matrix[i, j] = 0
    return noisy_matrix


Step 1: Load Attribute Item matrix.
Step 2: Get similarity (Jaccard) between attributes
Step 3: 

In [31]:
# Step 1: Example item–attribute dict
# Load item-attribute dict from MovieLens data
item_attr_dict = {}
with open('data/item_tag_dict.txt', 'r') as f:
    for line in f:
        movie_id, tags = line.strip().split(': ')
        item_attr_dict[movie_id] = tags.split(', ')

print(f"Loaded {len(item_attr_dict)} movies with their genres")
print("\nExample items:")
for movie_id, tags in list(item_attr_dict.items())[:5]:
    print(f"Movie {movie_id}: {tags}")

Mia, items, attrs = build_item_attribute_matrix(item_attr_dict)
attr_sim = jaccard_similarity(Mia)

Loaded 3883 movies with their genres

Example items:
Movie 1: ['animation', "children's", 'comedy']
Movie 2: ['adventure', "children's", 'fantasy']
Movie 3: ['comedy', 'romance']
Movie 4: ['comedy', 'drama']
Movie 5: ['comedy']


In [32]:
# Step 2: User–attribute preferences
Mua = generate_similarity_based_user_attributes(attr_sim, num_users=200, diversity_rounds=5)


In [33]:
# Step 3: Set-theoretic rules
overlaps = compute_attribute_overlap(Mia)
user_rules, conjunctions, negations = generate_set_theoretic_rules(overlaps, num_users=200, k=2, l=0, tau1=0.3, tau2=0.01, tau3=0.05)

In [67]:
# print a few user rules in attribute names parse the rules to attribute names

for rule in user_rules[:5]:
    and_clauses = [f"({attrs[a]} & {attrs[b]})" for (a, b) in rule['and']]
    not_clauses = [f"({attrs[present]} & not {attrs[not_present]})" for (present, not_present) in rule['not']]
    all_clauses = and_clauses + not_clauses
    print(" or ".join(all_clauses))

(adventure & comedy) or (animation & comedy) or (adventure & horror) or (comedy & sci-fi)
(crime & horror) or (thriller & war) or (action & western)
(crime & horror) or (adventure & horror) or (adventure & comedy) or (romance & not thriller)
(comedy & mystery) or (adventure & mystery)
(crime & mystery) or (comedy & fantasy) or (children's & sci-fi) or (action & thriller)


In [34]:
user_rules[12]

{'and': [(14, 15), (1, 5), (4, 5), (1, 12)], 'not': []}

In [35]:
# Step 4: User–item matrices
Mui_dot = generate_user_item_matrix_dot(Mua, Mia)
Mui_set = generate_user_item_matrix_set(user_rules, Mia)

# rank of Mui_dot
rank_Mui_dot = np.linalg.matrix_rank(Mui_dot)
print(f"Rank of Mui_dot: {rank_Mui_dot}")

# rank of Mui_set
rank_Mui_set = np.linalg.matrix_rank(Mui_set)
print(f"Rank of Mui_set: {rank_Mui_set}")


Rank of Mui_dot: 130
Rank of Mui_set: 188


In [36]:

def calculate_sparsity(matrix):
    """
    Calculate the sparsity of a matrix (percentage of zero elements).
    """
    total_elements = matrix.size
    zero_elements = np.count_nonzero(matrix == 0)
    sparsity = (zero_elements / total_elements) * 100
    return sparsity

# Calculate sparsity for both matrices
sparsity_dot = calculate_sparsity(Mui_dot)
sparsity_set = calculate_sparsity(Mui_set)

print(f"Sparsity of Mui_dot: {sparsity_dot:.2f}%")
print(f"Sparsity of Mui_set: {sparsity_set:.2f}%")
print(f"Number of non-zero elements in Mui_dot: {np.count_nonzero(Mui_dot)}")
print(f"Number of non-zero elements in Mui_set: {np.count_nonzero(Mui_set)}")
print(f"Total elements in matrix: {Mui_dot.size}")

Sparsity of Mui_dot: 90.26%
Sparsity of Mui_set: 93.79%
Number of non-zero elements in Mui_dot: 75663
Number of non-zero elements in Mui_set: 48243
Total elements in matrix: 776600


In [37]:
# Step 5: Combine preferences
Mui = combine_preferences(Mui_dot, Mui_set, p=0.6)

# Step 6: Inject noise
Mui_noisy = apply_noise(Mui, noise_fraction=0.1)

In [63]:
# sparsity of Mui, Mui_dot, Mui_set, Mui_noisy
print(f"Sparsity of Mui: {calculate_sparsity(Mui):.2f}%")
print(f"Sparsity of Mui_dot: {calculate_sparsity(Mui_dot):.2f}%")
print(f"Sparsity of Mui_set: {calculate_sparsity(Mui_set):.2f}%")
print(f"Sparsity of Mui_noisy: {calculate_sparsity(Mui_noisy):.2f}%")


Sparsity of Mui: 92.36%
Sparsity of Mui_dot: 90.26%
Sparsity of Mui_set: 93.79%
Sparsity of Mui_noisy: 93.12%


In [69]:
# check the rank of Mui, Mui_dot, Mui_set, Mui_noisy
print(f"Rank of Mui: {np.linalg.matrix_rank(Mui)}")
print(f"Rank of Mui_dot: {np.linalg.matrix_rank(Mui_dot)}")
print(f"Rank of Mui_set: {np.linalg.matrix_rank(Mui_set)}")
print(f"Rank of Mui_noisy: {np.linalg.matrix_rank(Mui_noisy)}")



Rank of Mui: 194
Rank of Mui_dot: 130
Rank of Mui_set: 188
Rank of Mui_noisy: 200


In [70]:
Mui_dot.shape

(200, 3883)

In [38]:

def train_test_split(matrix, test_ratio=0.2, seed=42):
    np.random.seed(seed)
    train = matrix.copy()
    test = np.zeros_like(matrix)

    for u in range(matrix.shape[0]):
        pos_items = np.where(matrix[u] == 1)[0]
        if len(pos_items) == 0:
            continue
        test_size = max(1, int(test_ratio * len(pos_items)))
        test_items = np.random.choice(pos_items, size=test_size, replace=False)
        train[u, test_items] = 0
        test[u, test_items] = 1

    return train, test

class TrainingDataset(Dataset):
    def __init__(self, user_item_matrix, num_negatives=1):
        self.num_users, self.num_items = user_item_matrix.shape
        self.positives = [(u, i) for u in range(self.num_users) for i in range(self.num_items) if user_item_matrix[u, i] == 1]
        self.negatives = []
        self.num_negatives = num_negatives
        self.user_item_matrix = user_item_matrix

    def __len__(self):
        return len(self.positives)

    def __getitem__(self, idx):
        u, i_pos = self.positives[idx]
        # positive example
        i_neg = np.random.randint(0, self.num_items)
        while self.user_item_matrix[u, i_neg] == 1:
            i_neg = np.random.randint(0, self.num_items)

        # randomly choose to return positive or negative sample
        if np.random.rand() < 0.5:
            return (
                torch.tensor(u, dtype=torch.long),
                torch.tensor(i_pos, dtype=torch.long),
                torch.tensor(1.0, dtype=torch.float)
            )
        else:
            return (
            torch.tensor(u, dtype=torch.long),
            torch.tensor(i_neg, dtype=torch.long),
            torch.tensor(0.0, dtype=torch.float)
            )


    
def evaluate_precision_hit(model, test_matrix, train_matrix, k=10):
    model.eval()
    num_users, num_items = test_matrix.shape
    precision_scores, hit_scores = [], []

    with torch.no_grad():
        for u in range(num_users):
            train_items = set(np.where(train_matrix[u] == 1)[0])
            test_items = set(np.where(test_matrix[u] == 1)[0])
            if not test_items:
                continue

            candidates = [i for i in range(num_items) if i not in train_items]
            user_tensor = torch.tensor([u] * len(candidates))
            item_tensor = torch.tensor(candidates)
            scores = model(user_tensor, item_tensor).numpy()

            top_k_indices = np.argsort(scores)[-k:][::-1]
            top_k_items = [candidates[i] for i in top_k_indices]

            hits = sum([1 for item in top_k_items if item in test_items])
            precision = hits / k
            hit = 1.0 if hits > 0 else 0.0

            precision_scores.append(precision)
            hit_scores.append(hit)

    avg_precision = np.mean(precision_scores)
    avg_hit = np.mean(hit_scores)
    print(f"Precision@{k}: {avg_precision:.4f}, Hit@{k}: {avg_hit:.4f}")


In [39]:
class MFModel(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=32):
        super(MFModel, self).__init__()
        self.user_emb = nn.Embedding(num_users, embedding_dim)
        self.item_emb = nn.Embedding(num_items, embedding_dim)
    
    def forward(self, user_idx, item_idx):
        u = self.user_emb(user_idx)
        i = self.item_emb(item_idx)
        dot = (u * i).sum(dim=1)
        return torch.sigmoid(dot)  # since preference is binary

In [40]:
def train(model, dataloader, epochs=5, loss_type='BCE', lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    if loss_type == 'BCE':
        loss_fn = nn.BCELoss()
    elif loss_type == 'MSE':
        loss_fn = nn.MSELoss()
    elif loss_type == 'NLL':
        loss_fn = nll
    else:
        raise ValueError(f"Invalid loss type: {loss_type}")
    
    for epoch in tqdm(range(epochs)):
        total_loss = 0.0
        model.train()
        for users, items, labels in dataloader:
            preds = model(users, items)
            if loss_type == 'BCE' or loss_type == 'MSE':
                loss = loss_fn(preds, labels)
            elif loss_type == 'NLL':
                # check if labels are 1 or 0
                pred_pos = preds[labels == 1]
                pred_neg = preds[labels == 0]
                loss = nll(pos=pred_pos, neg=pred_neg)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")
        evaluate_precision_hit(model, test_matrix, train_matrix, k=10)


In [56]:
train_matrix, test_matrix = train_test_split(Mui_set, test_ratio=0.2)

train_dataset = TrainingDataset(train_matrix, num_negatives=4)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

num_users, num_items = train_matrix.shape
model = MFModel(num_users, num_items, embedding_dim=64)


train(model, train_loader, loss_type='BCE', epochs=60, lr=0.005)
evaluate_precision_hit(model, test_matrix, train_matrix, k=10)

  0%|          | 0/60 [00:00<?, ?it/s]

Epoch 1, Loss: 1116.2058


  2%|▏         | 1/60 [00:01<01:41,  1.73s/it]

Precision@10: 0.0140, Hit@10: 0.1050
Epoch 2, Loss: 832.7717


  3%|▎         | 2/60 [00:03<01:36,  1.66s/it]

...

 98%|█████████▊| 59/60 [02:00<00:02,  2.02s/it]

Precision@10: 0.5815, Hit@10: 0.8950
Epoch 60, Loss: 6.7164


100%|██████████| 60/60 [02:02<00:00,  2.04s/it]

Precision@10: 0.5915, Hit@10: 0.8900


Precision@10: 0.5915, Hit@10: 0.8900


In [57]:
for k in [1, 5, 10]:
    evaluate_precision_hit(model, test_matrix, train_matrix, k=k)

Precision@1: 0.7250, Hit@1: 0.7250
Precision@5: 0.6370, Hit@5: 0.8550
Precision@10: 0.5915, Hit@10: 0.8900


In [58]:
train_matrix, test_matrix = train_test_split(Mui_dot, test_ratio=0.2)

train_dataset = TrainingDataset(train_matrix, num_negatives=4)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

num_users, num_items = train_matrix.shape
model = MFModel(num_users, num_items, embedding_dim=64)


train(model, train_loader, loss_type='BCE', epochs=60, lr=0.005)
evaluate_precision_hit(model, test_matrix, train_matrix, k=10)

  0%|          | 0/60 [00:00<?, ?it/s]

Epoch 1, Loss: 1702.1675


  2%|▏         | 1/60 [00:02<02:17,  2.33s/it]

Precision@10: 0.0205, Hit@10: 0.1800
Epoch 2, Loss: 1205.4785


  3%|▎         | 2/60 [00:04<02:09,  2.23s/it]

...

 98%|█████████▊| 59/60 [02:19<00:02,  2.46s/it]

Precision@10: 0.9670, Hit@10: 1.0000
Epoch 60, Loss: 19.0214


100%|██████████| 60/60 [02:21<00:00,  2.36s/it]

Precision@10: 0.9650, Hit@10: 1.0000


Precision@10: 0.9650, Hit@10: 1.0000


In [59]:
for k in [1, 5, 10]:
    evaluate_precision_hit(model, test_matrix, train_matrix, k=k)

Precision@1: 0.9900, Hit@1: 0.9900
Precision@5: 0.9780, Hit@5: 1.0000
Precision@10: 0.9650, Hit@10: 1.0000


In [53]:
# let's create the box intersection model
import torch
import torch.nn as nn
from box.box_wrapper import BoxTensor

class BoxIntersectionModel(nn.Module):
    def __init__(self,
                 num_users,
                 num_items,
                 embedding_dim=32,
                 intersection_temp=0.01,
                 volume_temp= 0.1):
        
        super().__init__()
        self.user_emb = nn.Embedding(num_users, 2 * embedding_dim)  # z and Z for each user
        self.item_emb = nn.Embedding(num_items, 2 * embedding_dim)  # z and Z for each item
        self.embedding_dim = embedding_dim
        self.intersection_temp = intersection_temp
        self.volume_temp = volume_temp

    def forward(self, user_idx, item_idx):
        # Get user and item box parameters
        user_box_params = self.user_emb(user_idx).view(-1, 2, self.embedding_dim)
        item_box_params = self.item_emb(item_idx).view(-1, 2, self.embedding_dim)
        user_box = BoxTensor(user_box_params)
        item_box = BoxTensor(item_box_params)
        # Compute intersection volume (as a score)
        intersection_vol = user_box.gumbel_intersection_log_volume(item_box, 
                                                                 volume_temp=self.volume_temp,
                                                                 intersection_temp=self.intersection_temp)
        item_vol = item_box.log_soft_volume_adjusted(volume_temp=self.volume_temp,
                                                      intersection_temp=self.intersection_temp)
        conditional_prob = intersection_vol - item_vol
        assert (conditional_prob <= 0).all(), "Log probability can not be positive"
        return conditional_prob

In [54]:
train_matrix, test_matrix = train_test_split(Mui_set, test_ratio=0.2)

train_dataset = TrainingDataset(train_matrix, num_negatives=5)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

num_users, num_items = train_matrix.shape
dim = 64
model = BoxIntersectionModel(num_users, num_items, embedding_dim=dim)
print(f"Dimension: {dim}")
train(model, train_loader, loss_type='NLL',epochs= 10 * int(np.log2(dim)) , lr=0.01)
evaluate_precision_hit(model, test_matrix, train_matrix, k=10)

Dimension: 64


  0%|          | 0/60 [00:00<?, ?it/s]

Epoch 1, Loss: 54814.9089


  2%|▏         | 1/60 [00:04<04:04,  4.14s/it]

Precision@10: 0.0960, Hit@10: 0.5000
Epoch 2, Loss: 24915.1379


...

 98%|█████████▊| 59/60 [04:30<00:04,  4.51s/it]

Precision@10: 0.7280, Hit@10: 0.9750
Epoch 60, Loss: 2116.4752


100%|██████████| 60/60 [04:34<00:00,  4.58s/it]

Precision@10: 0.7285, Hit@10: 0.9800


Precision@10: 0.7285, Hit@10: 0.9800


In [55]:
for k in [1, 5, 10]:
    evaluate_precision_hit(model, test_matrix, train_matrix, k=k)

Precision@1: 0.8050, Hit@1: 0.8050
Precision@5: 0.7820, Hit@5: 0.9550
Precision@10: 0.7285, Hit@10: 0.9800


In [60]:
train_matrix, test_matrix = train_test_split(Mui_dot, test_ratio=0.2)

train_dataset = TrainingDataset(train_matrix, num_negatives=4)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

num_users, num_items = train_matrix.shape
dim = 32
model = BoxIntersectionModel(num_users, num_items, embedding_dim=dim)
print(f"Dimension: {dim}")
train(model, train_loader, loss_type='NLL',epochs= 10 * int(np.log2(dim)) , lr=0.01)
evaluate_precision_hit(model, test_matrix, train_matrix, k=10)

Dimension: 32


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1, Loss: 36016.9535


  2%|▏         | 1/50 [00:03<03:09,  3.87s/it]

Precision@10: 0.5210, Hit@10: 0.9950
Epoch 2, Loss: 13519.7088


...

 98%|█████████▊| 49/50 [03:18<00:04,  4.01s/it]

Precision@10: 0.9090, Hit@10: 1.0000
Epoch 50, Loss: 14402.4844


100%|██████████| 50/50 [03:22<00:00,  4.06s/it]

Precision@10: 0.9150, Hit@10: 1.0000


Precision@10: 0.9150, Hit@10: 1.0000


In [61]:
for k in [1, 5, 10]:
    evaluate_precision_hit(model, test_matrix, train_matrix, k=k)

Precision@1: 0.9250, Hit@1: 0.9250
Precision@5: 0.9130, Hit@5: 0.9950
Precision@10: 0.9150, Hit@10: 1.0000


## Results

- Num Items 3883
- Num Users 200
- Num attributes 18
#### M_dot
- Mua @ Mia.T > tau (Mua: Generated, Mia: Real data)
- Mia is from real data.
- Mua is created by the common understanding 
    - "If you like this genre you might also like similar genre"
#### User rules

<style>
  .genre { 
    display: inline-block; 
    padding: 2px 6px; 
    margin: 1px; 
    border-radius: 6px; 
    font-size: 90%;
    font-weight: bold;
    color: white;
  }
  .adventure { background-color: #1f77b4; }
  .comedy { background-color: #ff7f0e; }
  .animation { background-color: #2ca02c; }
  .horror { background-color: #d62728; }
  .sci-fi { background-color: #9467bd; }
  .crime { background-color: #8c564b; }
  .thriller { background-color: #e377c2; }
  .war { background-color: #7f7f7f; }
  .action { background-color: #bcbd22; }
  .western { background-color: #17becf; }
  .romance { background-color: #ff9896; }
  .mystery { background-color: #c5b0d5; }
  .fantasy { background-color: #aec7e8; }
  .children { background-color: #98df8a; }
</style>

<p><strong>User 1:</strong><br>
(<span class="genre adventure">Adventure</span> ∧ <span class="genre comedy">Comedy</span>) ∨ 
(<span class="genre animation">Animation</span> ∧ <span class="genre comedy">Comedy</span>) ∨ 
(<span class="genre adventure">Adventure</span> ∧ <span class="genre horror">Horror</span>) ∨ 
(<span class="genre comedy">Comedy</span> ∧ <span class="genre sci-fi">Sci-Fi</span>)
</p>

<p><strong>User 2:</strong><br>
(<span class="genre crime">Crime</span> ∧ <span class="genre horror">Horror</span>) ∨ 
(<span class="genre thriller">Thriller</span> ∧ <span class="genre war">War</span>) ∨ 
(<span class="genre action">Action</span> ∧ <span class="genre western">Western</span>)
</p>

<p><strong>User 3:</strong><br>
(<span class="genre crime">Crime</span> ∧ <span class="genre horror">Horror</span>) ∨ 
(<span class="genre adventure">Adventure</span> ∧ <span class="genre horror">Horror</span>) ∨ 
(<span class="genre adventure">Adventure</span> ∧ <span class="genre comedy">Comedy</span>) ∨ 
(<span class="genre romance">Romance</span> ∧ ¬<span class="genre thriller">Thriller</span>)
</p>

<p><strong>User 4:</strong><br>
(<span class="genre comedy">Comedy</span> ∧ <span class="genre mystery">Mystery</span>) ∨ 
(<span class="genre adventure">Adventure</span> ∧ <span class="genre mystery">Mystery</span>)
</p>

<p><strong>User 5:</strong><br>
(<span class="genre crime">Crime</span> ∧ <span class="genre mystery">Mystery</span>) ∨ 
(<span class="genre comedy">Comedy</span> ∧ <span class="genre fantasy">Fantasy</span>) ∨ 
(<span class="genre children">Children's</span> ∧ <span class="genre sci-fi">Sci-Fi</span>) ∨ 
(<span class="genre action">Action</span> ∧ <span class="genre thriller">Thriller</span>)
</p>



Hypothesis
- M_set is not created by multiplication of matrices, so must be higher M_dot
- For M_set boxes would do better than vectors
- For M_dot vectors would do better than boxes (This we do not know for certain)
    - We do not know which angle we should be hitting for understanding the limitation with boxes.
    - Boxes might not be capturing low rank matrices, even if they do, it will be an overfitting to data.

<table border="1" cellpadding="5" cellspacing="0">
  <thead>
    <tr>
      <th rowspan="2">Matrix type</th>
      <th rowspan="2">Matrix rank</th>
      <th rowspan="2">Matrix sparsity</th>
      <th colspan="2">Vector</th>
      <th colspan="2">Box</th>
    </tr>
    <tr>
      <th>Precision@10</th>
      <th>Hits@10</th>
      <th>Precision@10</th>
      <th>Hits@10</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>M_dot</td>
      <td>130</td>
      <td>90.3</td>
      <td>96.5</td>
      <td>100.0</td>
      <td>91.5</td>
      <td>100.0</td>
    </tr>
    <tr>
      <td>M_set</td>
      <td>188</td>
      <td>93.8</td>
      <td>59.15</td>
      <td>89.0</td>
      <td>72.9</td>
      <td>98.0</td>
    </tr>
    <tr>
      <td>M_combined</td>
      <td>92.4</td>
      <td>194</td>
      <td>--</td>
      <td>--</td>
      <td>--</td>
      <td>--</td>
    </tr>
  </tbody>
</table>





Todo

[ ] M_combined: Right now just a convex sum. Evidently rank increases which is bad? Also need to run these experiments

[ ] Dataset size is small -- 1. Run k-fold cross validation 2. need to scale up.

[ ] Hyperparam search (Some manual tuning is done to make the training work)

[ ] Effect of dimension (current dimension size 64)

[ ] Disjunction = 0

[ ] how to combine -- disjucntion? or convex combination ?

[ ] more on the probing combination.. 

[ ] Size of the dataset might affect generalization... also the dimension of boxes? does it over fit.

[ ] training data size, rank, other meta parameters like the dimensions. 

[ ] Similar for the M_set.

[ ] 


[ ] Todo list


[ ] Check all the functionalities.
- Does the data generation agree with what I have? \\
- What is the rank / sparsity of these metrices?
- Is set is of higher rank?
- Also, study the spectrum of the PCA / SVD to come to the conclusion for ranks.
        Because the matrices might all be high rank because of the noisy or incompelteness. But they might be very close to a low rank, and we are after that approximation. Spectrum analysis would catch that


[ ] What is the best Eval in such scenario?
- Rec evals are more discrete so any poor model might also get hits@10 correct, or might just not get it at all. 
- Maybe RMSE is the way to go. Need to understand more on this.

[ ] check the dot product, does normalization help? Exhaustive search space.

[ ] How to densify the set-aspects.

[ ] Users are defined by latent factors.
 - Should those be boxes?
 - if so then what are the latent factors that are more favourable to boxes


[ ] 
- 

Possibilites
- Test bed for the latent box
- But if the higher dimensions are getting it, then honestly need to understand what is wrong.
